In [2]:
import numpy as np

def lagrange_global(x, y, x0):
    """
    Построение интерполяционного многочлена Лагранжа глобальным способом.

    Параметры:
        x (np.ndarray): Массив узлов интерполяции.
        y (np.ndarray): Массив значений функции в узлах.
        x0 (float): Точка, в которой нужно найти приближённое значение.

    Возвращает:
        float: Значение интерполяционного многочлена в точке x0.
    """
    n = len(x)
    L = 0.0
    for i in range(n):
        li = 1.0
        for j in range(n):
            if i != j:
                li *= (x0 - x[j]) / (x[i] - x[j])
        L += y[i] * li
    return L


def lagrange_piecewise(x, y, x0, k=3):
    """
    Кусочный метод Лагранжа: используем ближайшие k узлов для интерполяции.

    Параметры:
        x (np.ndarray): Массив узлов.
        y (np.ndarray): Массив значений функции.
        x0 (float): Точка интерполяции.
        k (int): Кол-во ближайших узлов.

    Возвращает:
        float: Приближённое значение в x0.
    """
    assert k <= len(x), "Недостаточно узлов"
    indices = np.argsort(np.abs(x - x0))[:k]
    return lagrange_global(x[indices], y[indices], x0)


def newton_forward_diff(x, y, x0):
    """
    Многочлен Ньютона на основе прямых конечных разностей (только для равномерной сетки).

    Параметры:
        x (np.ndarray): Узлы равномерной сетки.
        y (np.ndarray): Значения функции.
        x0 (float): Точка интерполяции.

    Возвращает:
        float: Приближённое значение в x0.
    """
    n = len(x)
    h = x[1] - x[0]
    assert np.allclose(np.diff(x), h), "Сетка должна быть равномерной"

    ### Таблица конечных разностей
    diff = np.zeros((n, n))
    diff[:, 0] = y

    for j in range(1, n):
        for i in range(n - j):
            diff[i, j] = diff[i+1, j-1] - diff[i][j-1]

    t = (x0 - x[0]) / h
    result = y[0]
    term = 1.0

    for i in range(1, n):
        term *= (t - i + 1) / i
        result += term * diff[0, i]

    return result


def newton_piecewise(x, y, x0, k=3):
    """
    Кусочная интерполяция многочленом Ньютона по ближайшим узлам (для равномерной сетки).

    Параметры:
        x (np.ndarray): Равномерная сетка.
        y (np.ndarray): Значения функции.
        x0 (float): Точка интерполяции.
        k (int): Число узлов для интерполяции.

    Возвращает:
        float: Значение приближённой функции в x0.
    """
    assert k <= len(x), "Недостаточно узлов"
    indices = np.argsort(np.abs(x - x0))[:k]
    sorted_indices = np.sort(indices)
    return newton_forward_diff(x[sorted_indices], y[sorted_indices], x0)


### Пример данных
x_uniform = np.array([0, 1, 2, 3], dtype=float)
y_uniform = np.array([1, 2, 1, 0], dtype=float)

x_nonuniform = np.array([0, 0.5, 2, 3.5], dtype=float)
y_nonuniform = np.array([1, 2, 1, 0], dtype=float)

x_interp = 1.5

### Вызовы
print("1. Глобальный Лагранж (равномерный):", lagrange_global(x_uniform, y_uniform, x_interp))
print("1. Глобальный Лагранж (неравномерный):", lagrange_global(x_nonuniform, y_nonuniform, x_interp))

print("2. Кусочный Лагранж (равномерный):", lagrange_piecewise(x_uniform, y_uniform, x_interp))
print("2. Кусочный Лагранж (неравномерный):", lagrange_piecewise(x_nonuniform, y_nonuniform, x_interp))

print("3. Глобальный Ньютон (равномерный):", newton_forward_diff(x_uniform, y_uniform, x_interp))
print("4. Кусочный Ньютон (равномерный):", newton_piecewise(x_uniform, y_uniform, x_interp))


1. Глобальный Лагранж (равномерный): 1.625
1. Глобальный Лагранж (неравномерный): 1.714285714285714
2. Кусочный Лагранж (равномерный): 1.75
2. Кусочный Лагранж (неравномерный): 2.0
3. Глобальный Ньютон (равномерный): 1.625
4. Кусочный Ньютон (равномерный): 1.75
